<a href="https://colab.research.google.com/github/keithferns98/ML_Models/blob/main/HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All Techniques Of Hyper Parameter 

Optimization

GridSearchCV

RandomizedSearchCV

Bayesian Optimization -Automate 

Hyperparameter Tuning (Hyperopt)

Sequential Model Based Optimization(Tuning 
a scikit-learn estimator with skopt)

Optuna- Automate Hyperparameter Tuning

Genetic Algorithms (TPOT Classifier)

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.shape

(768, 9)

In [4]:
df.tail()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [5]:
#df=df.dropna()

In [6]:
#df

In [7]:
import numpy as np
df['Glucose']=np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin']=np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness']=np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40,35.0,168.0,43.1,2.288,33,1


In [8]:
X=df.drop('Outcome',axis=1)
y=df['Outcome']

In [9]:
X.shape,y.shape

((768, 8), (768,))

In [10]:

#### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=0)

from sklearn.ensemble import RandomForestClassifier
rf_classifier=RandomForestClassifier(n_estimators=10).fit(X_train,y_train)
y_pred=rf_classifier.predict(X_test)
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [11]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [12]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[92 15]
 [19 28]]
0.7792207792207793
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       107
           1       0.65      0.60      0.62        47

    accuracy                           0.78       154
   macro avg       0.74      0.73      0.73       154
weighted avg       0.77      0.78      0.78       154



In [13]:
#Manual Hyperparameter Tuning

model=RandomForestClassifier(n_estimators=300,criterion='entropy',max_features='sqrt',min_samples_leaf=10,random_state=100).fit(X_train,y_train)
y_predmn=model.predict(X_test)
print(accuracy_score(y_test,y_predmn))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8246753246753247
[[92 15]
 [19 28]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       107
           1       0.65      0.60      0.62        47

    accuracy                           0.78       154
   macro avg       0.74      0.73      0.73       154
weighted avg       0.77      0.78      0.78       154



In [14]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
max_features=['auto','sqrt','log2']
max_depth=[int(x) for x in np.linspace(10,1000,10)]
min_samples_split=[1,2,3,4,5,7,9,10,14]
min_samples_leaf=[1,2,4,6,8,5,7,9,10,12]
random_grid={'n_estimators':n_estimators,
             'max_features':max_features,
             'max_depth':max_depth,
             'min_samples_split':min_samples_split,
             'min_samples_leaf':min_samples_leaf,
             'criterion':['gini','entropy']
             }
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 5, 7, 9, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8, 5, 7, 9, 10, 12], 'criterion': ['gini', 'entropy']}


In [15]:

rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,
                               cv=3,verbose=2,random_state=100,n_jobs=-1)
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   59.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [16]:
rf_randomcv.best_params_ 

{'criterion': 'gini',
 'max_depth': 340,
 'max_features': 'sqrt',
 'min_samples_leaf': 10,
 'min_samples_split': 3,
 'n_estimators': 400}

In [17]:
best_random_grid=rf_randomcv.best_estimator_

In [18]:
y_pred_rf=best_random_grid.predict(X_test)
print(accuracy_score(y_test,y_pred_rf))
print(confusion_matrix(y_test,y_pred_rf))
print(classification_report(y_test,y_pred))

0.8246753246753247
[[97 10]
 [17 30]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       107
           1       0.65      0.60      0.62        47

    accuracy                           0.78       154
   macro avg       0.74      0.73      0.73       154
weighted avg       0.77      0.78      0.78       154



GridSearchCV

In [19]:
rf_randomcv.best_params_

{'criterion': 'gini',
 'max_depth': 340,
 'max_features': 'sqrt',
 'min_samples_leaf': 10,
 'min_samples_split': 3,
 'n_estimators': 400}

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'criterion':[rf_randomcv.best_params_['criterion']],
    'max_depth':[rf_randomcv.best_params_['max_depth']],
    'max_features':[rf_randomcv.best_params_['max_features']],
    'min_samples_leaf':[rf_randomcv.best_params_['min_samples_leaf'],
                       rf_randomcv.best_params_['min_samples_leaf']+2,
                       rf_randomcv.best_params_['min_samples_leaf']+4],
    'n_estimators':[rf_randomcv.best_params_['n_estimators']-150,rf_randomcv.best_params_['n_estimators']-100,
                    rf_randomcv.best_params_['n_estimators'],
                    rf_randomcv.best_params_['n_estimators']+150 , rf_randomcv.best_params_['n_estimators']+100],

    'min_samples_split':[rf_randomcv.best_params_['min_samples_split']-2,
                         rf_randomcv.best_params_['min_samples_split']-1,
                         rf_randomcv.best_params_['min_samples_split'],
                         rf_randomcv.best_params_['min_samples_split']+1,
                         rf_randomcv.best_params_['min_samples_split']+2]
             
}
print(param_grid)

{'criterion': ['gini'], 'max_depth': [340], 'max_features': ['sqrt'], 'min_samples_leaf': [10, 12, 14], 'n_estimators': [250, 300, 400, 550, 500], 'min_samples_split': [1, 2, 3, 4, 5]}


In [21]:
rf_grid=RandomForestClassifier()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  6.1min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rand

In [22]:
best_pred=grid_search.best_estimator_

In [23]:
grid_search.best_params_

{'criterion': 'gini',
 'max_depth': 340,
 'max_features': 'sqrt',
 'min_samples_leaf': 14,
 'min_samples_split': 2,
 'n_estimators': 300}

In [24]:
y_pred_grid=best_pred.predict(X_test)
print(accuracy_score(y_test,y_pred_grid))
print(confusion_matrix(y_test,y_pred_grid))
print(classification_report(y_test,y_pred_grid))

0.8181818181818182
[[97 10]
 [18 29]]
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       107
           1       0.74      0.62      0.67        47

    accuracy                           0.82       154
   macro avg       0.79      0.76      0.77       154
weighted avg       0.81      0.82      0.81       154



Automated Hyperparameter Tuning can be done by using techniques such as

Bayesian Optimization

Gradient Descent

Evolutionary AlgorithmsAutomated Hyperparamter 

Tuning

1. Bayesian Optimization
Bayesian optimization uses probability to find the minimum of a function. The final aim is to find the input value to a function which can gives us the lowest possible output value.It usually performs better than random,grid and manual search providing better performance in the testing phase and reduced optimization time. In Hyperopt, Bayesian Optimization can be implemented giving 3 three main parameters to the function fmin.

Objective Function = defines the loss function to minimize.

Domain Space = defines the range of input values to test (in Bayesian Optimization this space creates a probability distribution for each of the used Hyperparameters).

Optimization Algorithm = defines the search algorithm to use to select the best input values to use in each new iteration

In [25]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials 


In [26]:
space={'criterion':hp.choice('criterion', ['entropy','gini']),
       'max_depth':hp.quniform('max_depth',10,1200,10), #prints 10 nos between 10-1200
       'max_features':hp.choice('max_features',['auto','log2','sqrt',None]),
       'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
       'min_samples_split':hp.uniform('min_samples_split',0,1),
       'n_estimators':hp.choice('n_estimators',[10,50,300,750,1200,1300,1500])
       }

In [27]:
space

{'criterion': <hyperopt.pyll.base.Apply at 0x7fa945d54fd0>,
 'max_depth': <hyperopt.pyll.base.Apply at 0x7fa945d543d0>,
 'max_features': <hyperopt.pyll.base.Apply at 0x7fa945d54490>,
 'min_samples_leaf': <hyperopt.pyll.base.Apply at 0x7fa945d542d0>,
 'min_samples_split': <hyperopt.pyll.base.Apply at 0x7fa945d54d10>,
 'n_estimators': <hyperopt.pyll.base.Apply at 0x7fa945d54610>}

In [30]:
space['criterion']

In [34]:
def objective(space):
  model=RandomForestClassifier(criterion=space['criterion'],max_depth=space['max_depth'],
                              max_features=space['max_features'],min_samples_leaf=space['min_samples_leaf'],
                              min_samples_split=space['min_samples_split'],n_estimators=space['n_estimators'])
  accuracy_hp=cross_val_score(model,X_train,y_train,cv=5).mean()
  return{'loss':-accuracy_hp,'status':STATUS_OK}

In [35]:
from sklearn.model_selection import cross_val_score
trials=Trials()
best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=100,trials=trials)
best

100%|██████████| 100/100 [11:20<00:00,  6.81s/it, best loss: -0.767093162734906]


{'criterion': 0,
 'max_depth': 940.0,
 'max_features': 1,
 'min_samples_leaf': 0.10439495227344749,
 'min_samples_split': 0.007334894862042279,
 'n_estimators': 4}

In [36]:

crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

entropy
sqrt
1200


In [38]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = 14, 
                                       min_samples_split = 2, 
                                       n_estimators = est[best['n_estimators']]).fit(X_train,y_train)
predictionforest = trainedforest.predict(X_test)
print(confusion_matrix(y_test,predictionforest))
print(accuracy_score(y_test,predictionforest))
print(classification_report(y_test,predictionforest))
acc5 = accuracy_score(y_test,predictionforest)

[[98  9]
 [21 26]]
0.8051948051948052
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       107
           1       0.74      0.55      0.63        47

    accuracy                           0.81       154
   macro avg       0.78      0.73      0.75       154
weighted avg       0.80      0.81      0.80       154



Genetic Algorithm

Genetic Algorithms tries to apply natural selection mechanisms to Machine Learning contexts.

Let's immagine we create a population of N Machine Learning models with some predifined Hyperparameters. We can then calculate the accuracy of each model and decide to keep just half of the models (the ones that performs best). We can now generate some offsprings having similar Hyperparameters to the ones of the best models so that go get again a population of N models. At this point we can again caltulate the accuracy of each model and repeate the cycle for a defined number of generations. In this way, just the best models will survive at the end of the process.

In [42]:
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
max_features=['auto','sqrt','log2']
min_samples_split=[2,5,10,14]
max_depth=[int(x) for x in np.linspace(10,1000,10) ]
min_samples_leaf=[1,2,4,6,8]
param={'n_estimators':n_estimators,
       'max_features':max_features,
       'max_depth':max_depth,
       'min_samples_leaf':min_samples_leaf,
       'min_samples_split':min_samples_split
    
}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_leaf': [1, 2, 4, 6, 8], 'min_samples_split': [2, 5, 10, 14]}


In [44]:
!pip install tpot


     |████████████████████████████████| 87 kB 2.7 MB/s 
     |████████████████████████████████| 160 kB 17.7 MB/s 
     |████████████████████████████████| 166.7 MB 5.7 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=3dce86fc05f5f206cc5fdfe9ad45d9206e5297c4b7ba9664560f86119467e6e0
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
from tpot import TPOTClassifier

In [46]:
tpot=TPOTClassifier(generations=5,population_size=24,offspring_size=12,
                    verbosity=2,early_stop=12,
                    config_dict={'sklearn.ensemble.RandomForestClassifier':param},
                    cv=4,scoring='accuracy')
tpot.fit(X_train,y_train)

Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7589657074951193

Generation 2 - Current best internal CV score: 0.7589657074951193

Generation 3 - Current best internal CV score: 0.7589657074951193

Generation 4 - Current best internal CV score: 0.7638358373652491

Generation 5 - Current best internal CV score: 0.7638358373652491

Best pipeline: RandomForestClassifier(RandomForestClassifier(input_matrix, max_depth=670, max_features=log2, min_samples_leaf=8, min_samples_split=14, n_estimators=1400), max_depth=1000, max_features=auto, min_samples_leaf=6, min_samples_split=2, n_estimators=1200)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                      890,
                                                                                      1000],
                                                                        'max_features': [

In [48]:
accuracy_tpot=tpot.score(X_test,y_test)

In [49]:
accuracy_tpot

0.8571428571428571

In [50]:
tpot.get_params

<bound method BaseEstimator.get_params of TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                      890,
                                                                                      1000],
                                               

In [52]:
!pip install optuna

     |████████████████████████████████| 302 kB 8.4 MB/s 
     |████████████████████████████████| 80 kB 6.1 MB/s 
     |████████████████████████████████| 208 kB 43.5 MB/s 
     |████████████████████████████████| 75 kB 3.3 MB/s 
     |████████████████████████████████| 141 kB 29.4 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
     |████████████████████████████████| 111 kB 42.4 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=2585042b6200535e93b5edd5cd04e9c42421b10c7f49abad03b2662a85a41348
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [53]:
import optuna

In [64]:
import sklearn.svm
def objective(trial):
  classifier=trial.suggest_categorical('classifier',['RandomForest','SVC'])
  if classifier=='RandomForest':
    n_estimators=trial.suggest_int('n_estimators',200,2000,10)
    max_depth=trial.suggest_int('max_depth',10,100,log=True)
    min_samples_leaf=trial.suggest_int('min_samples_leaf',2,20,10)
    clf=RandomForestClassifier(n_estimators=n_estimators,
                                      max_depth=max_depth,
                                      min_samples_leaf=min_samples_leaf)
  else:
      c=trial.suggest_float('svc_c',1e-10,1e10,log=True)
      clf=sklearn.svm.SVC(C=c,gamma=True)
  return cross_val_score(clf,X_train,y_train,n_jobs=-1,cv=3).mean()


In [65]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

trial=study.best_trial

print('Accuracy: {}'.format(trial.value))
print('Best Hyperparamter: {}'.format(trial.params))

[I 2021-09-03 10:37:35,120] A new study created in memory with name: no-name-af546aa3-fffe-4660-9d3e-c6c8f270333d
[I 2021-09-03 10:37:35,182] Trial 0 finished with value: 0.640068547744301 and parameters: {'classifier': 'SVC', 'svc_c': 1.0281941770678745e-10}. Best is trial 0 with value: 0.640068547744301.
[I 2021-09-03 10:37:37,218] Trial 1 finished with value: 0.7540411286465806 and parameters: {'classifier': 'RandomForest', 'n_estimators': 450, 'max_depth': 21, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.7540411286465806.
[I 2021-09-03 10:37:45,366] Trial 2 finished with value: 0.7507811254583134 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1910, 'max_depth': 11, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.7540411286465806.
[I 2021-09-03 10:37:50,881] Trial 3 finished with value: 0.752407141718476 and parameters: {'classifier': 'RandomForest', 'n_estimators': 1300, 'max_depth': 15, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.7540411

Accuracy: 0.7540570699824646
Best Hyperparamter: {'classifier': 'RandomForest', 'n_estimators': 1380, 'max_depth': 32, 'min_samples_leaf': 2}


In [66]:
trial



FrozenTrial(number=82, values=[0.7540570699824646], datetime_start=datetime.datetime(2021, 9, 3, 10, 44, 56, 910245), datetime_complete=datetime.datetime(2021, 9, 3, 10, 45, 3, 226145), params={'classifier': 'RandomForest', 'n_estimators': 1380, 'max_depth': 32, 'min_samples_leaf': 2}, distributions={'classifier': CategoricalDistribution(choices=('RandomForest', 'SVC')), 'n_estimators': IntUniformDistribution(high=2000, low=200, step=10), 'max_depth': IntLogUniformDistribution(high=100, low=10, step=1), 'min_samples_leaf': IntUniformDistribution(high=12, low=2, step=10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=82, state=TrialState.COMPLETE, value=None)

In [67]:
study.best_params

{'classifier': 'RandomForest',
 'max_depth': 32,
 'min_samples_leaf': 2,
 'n_estimators': 1380}

In [68]:
rf=RandomForestClassifier(n_estimators=1300,max_depth=32,min_samples_leaf=2)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=32, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [69]:
y_pred_opt=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred_opt))
print(accuracy_score(y_test,y_pred_opt))
print(classification_report(y_test,y_pred))

[[96 11]
 [16 31]]
0.8246753246753247
              precision    recall  f1-score   support

           0       0.83      0.86      0.84       107
           1       0.65      0.60      0.62        47

    accuracy                           0.78       154
   macro avg       0.74      0.73      0.73       154
weighted avg       0.77      0.78      0.78       154

